In [1]:
from utils import tokenize, load_curpus
import numpy as np

#### 加载数据

In [2]:
import pandas as pd
train_data = load_curpus("weibo2018/train.txt")
test_data = load_curpus("weibo2018/test.txt")
train_df = pd.DataFrame(train_data, columns=["content", "sentiment"])
test_df = pd.DataFrame(test_data, columns=["content", "sentiment"])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/dy/xjy0y7v97js5x1bghby2fnkm0000gn/T/jieba.cache
Loading model cost 0.702 seconds.
Prefix dict has been built succesfully.


In [3]:
stopwords = []
with open("stopwords.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())

#### Ont-hot
ps: 与其他方法相比，朴素贝叶斯并没有对高质量词向量的需求，因此不再加载Fasttext词向量，而是直接用one-hot形式

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
data_str = [" ".join(content) for content, sentiment in train_data] + \
            [" ".join(content) for content, sentiment in test_data]
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
vectorizer.fit_transform(data_str)

<10500x38718 sparse matrix of type '<class 'numpy.int64'>'
	with 172234 stored elements in Compressed Sparse Row format>

In [5]:
X_data, y_data = [], []
for content, sentiment in train_data:
    X, y = [], sentiment
    X_data.append(" ".join(content))
    y_data.append(sentiment)
X_train = vectorizer.transform(X_data)
y_train = y_data

In [6]:
X_data, y_data = [], []
for content, sentiment in test_data:
    X, y = [], sentiment
    X_data.append(" ".join(content))
    y_data.append(sentiment)
X_test = vectorizer.transform(X_data)
y_test = y_data

#### Bayes
全部用默认参数

In [7]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [8]:
result = clf.predict(X_test)

#### 模型评估
速度快，效果还很好，可能是因为该任务语料规模较小，在大规模语料任务上性能会下降

In [9]:
from sklearn import metrics
print(metrics.classification_report(y_test, result))
print("准确率:", metrics.accuracy_score(y_test, result))

             precision    recall  f1-score   support

          0       0.73      0.86      0.79       155
          1       0.93      0.86      0.89       345

avg / total       0.87      0.86      0.86       500

准确率: 0.858
